In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
%matplotlib widget
from ipywidgets import interact, fixed

In [45]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [46]:
from ecephys.sglx_utils import load_timeseries
from ecephys.scoring import load_visbrain_hypnogram, mask_times

In [47]:
from ecephys.signal.ripples import make_speed_vector, apply_ripple_filter, generalized_Kay_ripple_detector, compute_ripple_features
from ecephys.signal.timefrequency import get_avg_perievent_cwtm

In [48]:
from ecephys.plot import plot_all_ripples, plot_ripple

In [49]:
from neurodsp.plts import plot_time_series

## Load the data

In [50]:
bin_path = Path('/Volumes/neuropixel/Data/CNPIX4-Doppio/3-18-2020_g0_t3.imec0.lf.bin') # Doppio SR
#bin_path = Path('/Volumes/neuropixel/Data/CNPIX6-Eugene/9.24.2020_SR_24hs_g0_t0.imec0.lf.bin') # Eugene SR

In [51]:
chans = np.asarray([161, 162, 165, 166, 169, 170, 173, 174, 177, 178, 181]) # Doppio
# chans = np.asarray([162, 165, 166, 169, 170, 173, 174, 177, 178, 181, 182]) # Valentino
# chans = np.asarray([159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179]) # Alessandro
# chans = np.asarray([163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183]) #Segundo
# chans = np.asarray([193, 195, 197, 199, 201, 203, 205, 207, 209, 211]) #Eugene

In [52]:
(time, lfps, sampling_frequency) = load_timeseries(bin_path, chans, start_time=None, end_time=None)

nChan: 385, nFileSamp: 18000019


In [ ]:
lfps_df = pd.DataFrame(lfps, columns=chans, index=pd.Index(time, name='time'))
lfps_df.plot(subplots=True, figsize=(18, 6))
plt.show()

## Load hypnogram and select only NREM epochs

In [53]:
hypno_path = Path('/Volumes/neuropixel/Data/CNPIX4-Doppio/3-18-2020_g0_t3.hypnogram.txt') # Doppio SR
# hypno_path = Path('/Volumes/neuropixel/Data/CNPIX6-Eugene/9.24.2020_SR_24hs_g0_t0.hypnogram.txt') # Eugene SR

In [54]:
hypno = load_visbrain_hypnogram(hypno_path)

In [55]:
speed = make_speed_vector(hypno, ['Wake', 'N1', 'N2', 'REM'], time / 1000)

## Apply ripple bandpass filter

In [56]:
filtered_lfps = apply_ripple_filter(lfps, sampling_frequency)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 6))
ax1.plot(time, lfps)
ax2.plot(time, filtered_lfps)
plt.show()

## Detect Ripples

### Kay method
Detection of SWRs was prerequisite for all data analyzed in this study, and was performed only when at least three CA1 cell layer recordings were available. Offline, a multisite average approach was used to detect SWRs57. Specifically, LFPs from all available CA1 cell layer tetrodes were filtered between 150–250 Hz, then squared and summed across tetrodes. This sum was smoothed with a Gaussian kernel (σ = 4 ms) and the square root of the smoothed sum was analyzed. SWRs were detected when the signal exceeded 2 s.d. of the recording epoch mean for at least 15 ms. SWR periods were then defined as the periods, containing the times of threshold crossing, in which the power trace exceeded the mean. SWR onset was defined as the start of a SWR period. Detection of SWRs was performed only when subjects' head speed was <4 cm/s. For SWR-triggered spike raster plots and PSTH plots, a 0.5 s exclusion period was imposed to isolate SWRs occurring only after non-SWR periods; otherwise, analyses of SWRs included all detected SWRs.

In [34]:
from ripple_detection import Kay_ripple_detector

Kay_ripple_times = Kay_ripple_detector(time, filtered_lfps, speed, sampling_frequency, zscore_threshold=3)

In [ ]:
plot_all_ripples(time, lfps, filtered_lfps, Kay_ripple_times)

### Karlsson method
Sharp waves ripple events (SWRs) were identified based on peaks in the local field potential (LFP) recorded from one channel from each tetrode in the CA3 and CA1 cell layers. The raw LFP data were band-pass filtered between 150–250 Hz and the SWR envelope was determined using a Hilbert transform. The envelope was smoothed with a Gaussian (4 ms stdev). We initially identified SWR events as sets of times when the smoothed envelope stayed above 3 standard deviations of the mean for at least 15 ms on at least one tetrode. We defined the entire SWR as including times immediately before and after that threshold crossing event during which the envelope exceeded the mean. Overlapping SWRs were combined across tetrodes, so many events extended beyond the SWR seen on a single tetrode.

In [ ]:
from ripple_detection import Karlsson_ripple_detector

Karlsson_ripple_times = Karlsson_ripple_detector(time, filtered_lfps, speed, sampling_frequency)

### Eschenko-like method
SPW-Rs were detected by means of an automatic thresholding algorithm (Siapas and Wilson 1998; Csicsvari et al. 1999), illustrated in Figure 1. The hippocampal LFP signal was first filtered (150–250 Hz) and the root mean square (RMS) was calculated at every 5 msec (in a moving 10-msec window). The threshold for ripple peak detection was set to 4 SDs above the mean RMS signal. The beginning and end of a ripple were marked at points at which the RMS signal dropped below 2 SD provided that these two points were separated by 25 msec; these time points were used to estimate ripple duration. For every marked ripple, the troughs of the ripples were detected as the minima of the filtered LFP signal, and the deepest trough was marked as the time point representing the respective SPW-R event. The magnitude of the ripple was determined by integrating the RMS signal over the duration of each ripple (King et al. 1999). Ripples were detected during SWS periods only. The ripple amplitude was estimated as a voltage difference between maximum and minimum peaks of the filtered CA1 signal during the ripple event. Ripple density was defined as the number of ripples/second. Figure 1A shows the raw data from a hippocampal electrode. Ripple parameters and how they were measured are illustrated in Figure 1B.

In [57]:
Eschenko_ripple_times = generalized_Kay_ripple_detector(time, filtered_lfps, speed, sampling_frequency, detection_zscore_threshold=4.0, boundary_zscore_threshold=1.0, minimum_duration=0.025)

## Collect ripple statistics

In [58]:
ripple_times = Eschenko_ripple_times
ripple_features = compute_ripple_features(time, filtered_lfps, ripple_times, sampling_frequency, 'Kay')

## Export results

In [59]:
# Export the results for use in MATLAB. 
# results_file = Path('/Volumes/neuropixel/Data/CNPIX3-Valentino/2-20-2020_g0_t3.N2_ripples.csv')
results_file = Path('/Volumes/neuropixel/Data/CNPIX4-Doppio/3-18-2020_g0_t3.all_ripples.csv')
# results_file = Path('/Volumes/neuropixel/Data/CNPIX6-Eugene/9.24.2020_SR_24hs_g0_t0.N2_ripples.csv')
ripple_features.to_csv(results_file)

[autoreload of ecephys.signal.ripples failed: Traceback (most recent call last):
  File "/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name 'unnest_df' from 'ecephys.helpers' (/home/gfindlay/Documents/ecephys/ecephys/helpers/__init__.py)
]


## Explore results

In [60]:
_, ax = plt.subplots(3, 1, figsize=(18, 6))
_ = interact(
    plot_ripple,
    time=fixed(time),
    lfps=fixed(lfps),
    filtered_lfps=fixed(filtered_lfps),
    ripple_times=fixed(ripple_times),
    window_length=(0.25, 2, 0.25),
    ripple_number=(1, len(ripple_times), 1),
    ax=fixed(ax),
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='window_length', max=2.0, min=0.25, step=0.25), IntSl…